In [ ]:
import geopandas as gpd
import pandas as pd
from osgeo import ogr,gdal
import os
import xarray as xr
import numpy as np
import pyproj
from pygeos import from_wkb,from_wkt
import pygeos
from tqdm import tqdm
from shapely.wkb import loads
from pathlib import Path

In [ ]:
gdal.SetConfigOption("OSM_CONFIG_FILE", os.path.join('..',"osmconf.ini"))

# change paths to make it work on your own machine
data_path = os.path.join('C:\\','data','pg_risk_analysis')
tc_path = os.path.join(data_path,'tc_netcdf')
osm_data_path = os.path.join('C:\\','data','country_osm')

In [ ]:
def query_b(geoType,keyCol,**valConstraint):
    """
    This function builds an SQL query from the values passed to the retrieve() function.
    Arguments:
         *geoType* : Type of geometry (osm layer) to search for.
         *keyCol* : A list of keys/columns that should be selected from the layer.
         ***valConstraint* : A dictionary of constraints for the values. e.g. WHERE 'value'>20 or 'value'='constraint'
    Returns:
        *string: : a SQL query string.
    """
    query = "SELECT " + "osm_id"
    for a in keyCol: query+= ","+ a  
    query += " FROM " + geoType + " WHERE "
    # If there are values in the dictionary, add constraint clauses
    if valConstraint: 
        for a in [*valConstraint]:
            # For each value of the key, add the constraint
            for b in valConstraint[a]: query += a + b
        query+= " AND "
    # Always ensures the first key/col provided is not Null.
    query+= ""+str(keyCol[0]) +" IS NOT NULL" 
    return query 


def retrieve(osm_path,geoType,keyCol,**valConstraint):
    """
    Function to extract specified geometry and keys/values from OpenStreetMap
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.     
        *geoType* : Type of Geometry to retrieve. e.g. lines, multipolygons, etc.
        *keyCol* : These keys will be returned as columns in the dataframe.
        ***valConstraint: A dictionary specifiying the value constraints.  
        A key can have multiple values (as a list) for more than one constraint for key/value.  
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all columns, geometries, and constraints specified.    
    """
    driver=ogr.GetDriverByName('OSM')
    data = driver.Open(osm_path)
    query = query_b(geoType,keyCol,**valConstraint)
    sql_lyr = data.ExecuteSQL(query)
    features =[]
    # cl = columns 
    cl = ['osm_id'] 
    for a in keyCol: cl.append(a)
    if data is not None:
        print('query is finished, lets start the loop')
        for feature in tqdm(sql_lyr,desc='extract'):
            #try:
            if feature.GetField(keyCol[0]) is not None:
                geom1 = (feature.geometry().ExportToWkt())
                #print(geom1)
                geom = from_wkt(feature.geometry().ExportToWkt()) 
                if geom is None:
                    continue
                # field will become a row in the dataframe.
                field = []
                for i in cl: field.append(feature.GetField(i))
                field.append(geom)   
                features.append(field)
            #except:
            #    print("WARNING: skipped OSM feature")   
    else:
        print("ERROR: Nonetype error when requesting SQL. Check required.")    
    cl.append('geometry')                   
    if len(features) > 0:
        return pd.DataFrame(features,columns=cl)
    else:
        print("WARNING: No features or No Memory. returning empty GeoDataFrame") 
        return pd.DataFrame(columns=['osm_id','geometry'])

def power_polyline(osm_path):
    """
    Function to extract all energy linestrings from OpenStreetMap  
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with specified unique energy linestrings.
    """
    df = retrieve(osm_path,'lines',['power','voltage'])
    
    df = df.reset_index(drop=True).rename(columns={'power': 'asset'})
   
    return df.reset_index(drop=True)

def power_polygon(osm_path): # check with joel, something was wrong here with extracting substations
    """
    Function to extract energy polygons from OpenStreetMap  
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with specified unique energy linestrings.
    """
    df = retrieve(osm_path,'multipolygons',['other_tags']) 
    
    df = df.loc[(df.other_tags.str.contains('power'))]   #keep rows containing power data         
    df = df.reset_index(drop=True).rename(columns={'other_tags': 'asset'})     
    
    df['asset'].loc[df['asset'].str.contains('"power"=>"substation"', case=False)]  = 'substation' #specify row
    df['asset'].loc[df['asset'].str.contains('"power"=>"plant"', case=False)] = 'plant' #specify row
    
    df = df.loc[(df.asset == 'substation') | (df.asset == 'plant')]
            
    return df.reset_index(drop=True) 

def electricity(osm_path):
    """
    Function to extract building polygons from OpenStreetMap    
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all unique building polygons.    
    """
    df = retrieve(osm_path,'multipolygons',['power'])
    
    df = df.reset_index(drop=True).rename(columns={'power': 'asset'})
    
    df = df[df.asset!='generator']
    df['asset'].loc[df['asset'].str.contains('"power"=>"substation"', case=False)]  = 'substation' #specify row
    df['asset'].loc[df['asset'].str.contains('"power"=>"plant"', case=False)] = 'plant' #specify row
    
    df = df.loc[(df.asset == 'substation') | (df.asset == 'plant')]
    
    return df.reset_index(drop=True)

def retrieve_poly_subs(osm_path, w_list, b_list):
    """
    Function to extract electricity substation polygons from OpenStreetMap
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region
        for which we want to do the analysis.
        *w_list* :  white list of keywords to search in the other_tags columns
        *b_list* :  black list of keywords of rows that should not be selected
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with specified unique substation.
    """
    df = retrieve(osm_path,'multipolygons',['other_tags'])
    df = df[df.other_tags.str.contains('substation', case=False, na=False)]
    #df = df.loc[(df.other_tags.str.contains('substation'))]
    df = df[~df.other_tags.str.contains('|'.join(b_list))]
    #df = df.reset_index(drop=True).rename(columns={'other_tags': 'asset'})
    df['asset']  = 'substation' #specify row
    #df = df.loc[(df.asset == 'substation')] #specify row
    return df.reset_index(drop=True)

def power_point(osm_path):
    """
    Function to extract energy points from OpenStreetMap  
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with specified unique energy linestrings.
    """   
    df = retrieve(osm_path,'points',['other_tags']) 
    df = df.loc[(df.other_tags.str.contains('power'))]  #keep rows containing power data       
    df = df.reset_index(drop=True).rename(columns={'other_tags': 'asset'})     
    print(df)
    
    df['asset'].loc[df['asset'].str.contains('"power"=>"tower"', case=False)]  = 'power_tower' #specify row
    df['asset'].loc[df['asset'].str.contains('"power"=>"pole"', case=False)] = 'power_pole' #specify row
    df['asset'].loc[df['asset'].str.contains('"utility"=>"power"', case=False)] = 'power_tower' #specify row
    
    df = df.loc[(df.asset == 'power_tower') | (df.asset == 'power_pole')]
            
    return df.reset_index(drop=True)

In [ ]:
power_point(osm_path)

In [ ]:
def reproject(df_ds,current_crs="epsg:4326",approximate_crs = "epsg:3857"):
    """[summary]

    Args:
        df_ds ([type]): [description]
        current_crs (str, optional): [description]. Defaults to "epsg:3857".
        approximate_crs (str, optional): [description]. Defaults to "epsg:4326".

    Returns:
        [type]: [description]
    """    

    geometries = df_ds['geometry']
    coords = pygeos.get_coordinates(geometries)
    transformer=pyproj.Transformer.from_crs(current_crs, approximate_crs,always_xy=True)
    new_coords = transformer.transform(coords[:, 0], coords[:, 1])
    
    return pygeos.set_coordinates(geometries.copy(), np.array(new_coords).T) 

def load_curves_maxdam(data_path,hazard='wind'): 
    """[summary]

    Args:
        data_path ([type]): [description]

    Returns:
        [type]: [description]
    """
    
    if hazard == 'wind':
        sheet_name = 'flooding_curves'
    elif hazard == 'flood':
        sheet_name = 'flooding_curves'
    
    # load curves and maximum damages as separate inputs
    curves = pd.read_excel(data_path,sheet_name=sheet_name,skiprows=8,index_col=[0])
    maxdam=pd.read_excel(data_path,sheet_name=sheet_name,index_col=[0]).iloc[:5]
    
    curves.columns = maxdam.columns

    #transpose maxdam so its easier work with the dataframe
    maxdam = maxdam.T

    #interpolate the curves to fill missing values
    curves = curves.interpolate()
   
    return curves,maxdam

def buffer_assets(assets,buffer_size=100):
    """[summary]

    Args:
        assets ([type]): [description]
        buffer_size (int, optional): [description]. Defaults to 100.

    Returns:
        [type]: [description]
    """    
    assets['buffered'] = pygeos.buffer(assets.geometry.values,buffer_size)
    return assets

def overlay_hazard_assets(df_ds,assets):
    """[summary]

    Args:
        df_ds ([type]): [description]
        assets ([type]): [description]

    Returns:
        [type]: [description]
    """
    #overlay 
    hazard_tree = pygeos.STRtree(df_ds.geometry.values)
    if (pygeos.get_type_id(assets.iloc[0].geometry) == 3) | (pygeos.get_type_id(assets.iloc[0].geometry) == 6):
        return  hazard_tree.query_bulk(assets.geometry,predicate='intersects')    
    else:
        return  hazard_tree.query_bulk(assets.buffered,predicate='intersects')
    
def get_damage_per_asset_per_rp(asset,df_ds,assets,curves,maxdam,return_period,country):
    """[summary]

    Args:
        asset ([type]): [description]
        df_ds ([type]): [description]
        assets ([type]): [description]
        grid_size (int, optional): [description]. Defaults to 90.

    Returns:
        [type]: [description]
    """    

    # find the exact hazard overlays:
    get_hazard_points = df_ds.iloc[asset[1]['hazard_point'].values].reset_index()
    get_hazard_points = get_hazard_points.loc[pygeos.intersects(get_hazard_points.geometry.values,assets.iloc[asset[0]].geometry)]

    asset_type = assets.iloc[asset[0]].asset
    asset_geom = assets.iloc[asset[0]].geometry

    if asset_type in ['plant','substation','generator']:
        maxdam_asset = maxdam.loc[asset_type].MaxDam/pygeos.area(asset_geom)
    else:
        maxdam_asset = maxdam.loc[asset_type].MaxDam


    hazard_intensity = curves[asset_type].index.values
    fragility_values = curves[asset_type].values
    
    if len(get_hazard_points) == 0:
        return asset[0],0
    else:
        
        if pygeos.get_type_id(asset_geom) == 1:
            get_hazard_points['overlay_meters'] = pygeos.length(pygeos.intersection(get_hazard_points.geometry.values,asset_geom))
            return asset[0],np.sum((np.interp(get_hazard_points[return_period].values,hazard_intensity,fragility_values))*get_hazard_points.overlay_meters*maxdam_asset)
        
        elif  pygeos.get_type_id(asset_geom) == 3:
            get_hazard_points['overlay_m2'] = pygeos.area(pygeos.intersection(get_hazard_points.geometry.values,asset_geom))
            return asset[0],get_hazard_points.apply(lambda x: np.interp(x[return_period], hazard_intensity, fragility_values)*maxdam_asset*x.overlay_m2,axis=1).sum()     
        
        else:
            return asset[0],np.sum((np.interp(get_hazard_points[return_period].values,hazard_intensity,fragility_values))*maxdam_asset)

In [ ]:
def open_hazard_data(climate_model,hazard_type='storm'):
    
    if hazard_type == 'storm':
        with xr.open_dataset(os.path.join(tc_path,'STORM_FIXED_RETURN_PERIODS{}_WP.nc'.format(climate_model))) as ds:
            """
            TC climate model:
                CMCC-CM2-VHR4
                CNRM-CM6-1-HR
                EC-Earth3P-HR
                HadGEM3-GC31-HM
            """
            print(ds.keys())
            
            # get the mean values
            df_ds = ds['mean'].to_dataframe().unstack(level=2).reset_index()

            # create geometry values and drop lat lon columns
            df_ds['geometry'] = [pygeos.points(x) for x in list(zip(df_ds['lon'],df_ds['lat']))]
            df_ds = df_ds.drop(['lat','lon'],axis=1,level=0)

            #rename columns to return periods
            return_periods = ['1_{}{}'.format(int(x),climate_model) for x in ds['rp']]
            df_ds.columns = ['1_{}{}'.format(int(x),climate_model) for x in list(df_ds.columns.get_level_values(1))[:-1]]+['geometry']     
            df_ds['geometry'] = pygeos.buffer(df_ds.geometry,radius=0.1/2,cap_style='square').values
            df_ds['geometry'] = reproject(df_ds)
            
            # drop all non values to reduce size
            if climate_model == '':
                df_ds = df_ds.loc[~df_ds['1_10000'].isna()].reset_index(drop=True)
            
            df_ds = df_ds.fillna(0)
                        
    elif hazard_type == 'flood':
        
        # THIS STILL NEEDS TO BE TESTED WITH GLOFRIS DATA
        with xr.open_dataset(os.path.join(fl_path,'HIST/inuncoast_historical_nosub_hist_rp0500_0.nc')) as ds: #, engine="rasterio"
            df_ds = ds.to_dataframe().reset_index()
            df_ds['geometry'] = pygeos.points(df_ds.x,y=df_ds.y)
            df_ds = df_ds.rename(columns={'band_data': 'hazard_intensity'})
            df_ds = df_ds.drop(['band','x', 'y','spatial_ref'], axis=1)
            df_ds = df_ds.dropna()
            df_ds = df_ds.reset_index(drop=True)
            df_ds.geometry= pygeos.buffer(df_ds.geometry,radius=20/2,cap_style='square').values
            df_ds['geometry'] = reproject(df_ds)
        
    return df_ds

In [ ]:
osm_data_path = os.path.join('C:\\','data','country_osm')
country_code = 'VNM'
osm_path = os.path.join(osm_data_path,'{}.osm.pbf'.format(country_code))

In [ ]:
%%time
# load hazard data 
climate_model = '_CMCC-CM2-VHR4' #_CMCC-CM2-VHR4
df_ds = open_hazard_data(climate_model, hazard_type='storm')
df_ds.head()
#df_ds.to_csv(os.path.join(data_path,'retrieve_data/df_ds.csv'))

In [ ]:
osm_data_path = os.path.join('C:\\','data','country_osm')
filelist = []
country_codes = []
for i in os.listdir(osm_data_path):
    osm_path = os.path.join(osm_data_path,i)
    if os.path.isfile(osm_path):
        filelist.append(i)
        country_codes.append(os.path.splitext(os.path.splitext(i)[0])[0])
        print(country_codes)
        osm_path = os.path.join(osm_data_path,i)
        print(osm_path)
country_codes = tuple(country_codes)
country_codes[0]

In [ ]:
# extract all data from OSM for power infrastructure
# reproject to epsg:3857 to have coordinate system in meters
#osm_path = os.path.join(osm_data_path,'{}.osm.pbf'.format(country_code))

country_code = ''

#lines
power_lines = {country_code:[]}
for country_code in country_codes:
    osm_path = os.path.join(osm_data_path,'{}.osm.pbf'.format(country_code))
    print(osm_path)
    power_lines_country = power_polyline(osm_path)
    power_lines_country['geometry'] = reproject(power_lines_country)
    power_lines_country = buffer_assets(power_lines_country.loc[power_lines_country.asset.isin(
        ['cable','minor_cable','line','minor_line'])],buffer_size=100).reset_index(drop=True)
    power_lines[country_code] = power_lines_country

#polygons
power_poly = {country_code:[]}
for country_code in country_codes:
    osm_path = os.path.join(osm_data_path,'{}.osm.pbf'.format(country_code))
    print(osm_path)
    power_poly_country = electricity(osm_path)
    power_poly_country['geometry'] = reproject(power_poly_country)
    power_poly[country_code] = power_poly_country
    
""" 
    power_poly_country = power_polygon(osm_path)
    power_poly_country['geometry'] = reproject(power_poly_country)
    #power_poly = power_poly.loc[power_poly.asset.isin(['plant','substation'])].reset_index(drop=True)
    power_poly[country_code] = power_poly_country
   
    #substations polygons
    ### KEY:VAL IN OTHER_TAGS ###
    w_list = ['substation'] # add in funtion '|'.join(w_list) if more than one key word
    b_list = ['minor_distribution', 'converter', 'indoor', 'pipeline', 'gas']
    power_sub = retrieve_poly_subs(osm_path,w_list,b_list)
    #power_sub = retrieve_poly_subs.rename(columns={'power':'asset'})
    power_sub['geometry'] = reproject(power_sub)
"""
#points
power_points = {country_code:[]}
for country_code in country_codes:
    osm_path = os.path.join(osm_data_path,'{}.osm.pbf'.format(country_code))
    print(osm_path)
    power_points_country = power_point(osm_path)
    power_points_country['geometry'] = reproject(power_points_country)
    power_points_country = buffer_assets(power_points_country.loc[power_points_country.asset.isin(
        ['power_tower','power_pole'])],buffer_size=100).reset_index(drop=True)
    power_points[country_code] = power_points_country

In [ ]:
# load curves and maxdam
curves,maxdam = load_curves_maxdam(data_path=os.path.join('..','data','infra_vulnerability_data.xlsx'))
curves['line'] = 1 # remove this when things work!

In [ ]:
# return periods
return_periods = ['1_10{}'.format(climate_model),'1_50{}'.format(climate_model),'1_100{}'.format(climate_model),'1_1000{}'.format(climate_model)]

In [ ]:
# lines in loop
damaged_lines = {country_code:[]}
for country_code in country_codes:
    overlay_lines = pd.DataFrame(overlay_hazard_assets(df_ds,power_lines.get(country_code)).T,columns=['asset','hazard_point'])
    collect_line_damages = []
    for asset in tqdm(overlay_lines.groupby('asset'),total=len(overlay_lines.asset.unique()),desc='polyline damage calculation for {}'.format(country_code)):
        for return_period in return_periods:
            collect_line_damages.append([return_period,get_damage_per_asset_per_rp(asset,df_ds,power_lines.get(country_code),curves,maxdam,return_period,country_code)])

    collect_line_damages = [(line[0],line[1][0],line[1][1]) for line in collect_line_damages]
    damaged_lines_country = power_lines.get(country_code).merge(pd.DataFrame(collect_line_damages,columns=['return_period','index','damage']),left_index=True,right_on='index')
    damaged_lines_country = damaged_lines_country.drop(['buffered'],axis=1)
    damaged_lines[country_code] = damaged_lines_country
#damaged_lines

In [ ]:
damaged_lines

In [ ]:
# polygons in loop
damaged_poly = {country_code:[]}
for country_code in country_codes:
    overlay_poly = pd.DataFrame(overlay_hazard_assets(df_ds,power_poly.get(country_code)).T,columns=['asset','hazard_point'])
    collect_poly_damages = []
    for asset in tqdm(overlay_poly.groupby('asset'),total=len(overlay_poly.asset.unique()),desc='polygon damage calculation for {}'.format(country_code)):
        for return_period in return_periods:
            collect_poly_damages.append([return_period,get_damage_per_asset_per_rp(asset,df_ds,power_poly.get(country_code),curves,maxdam,return_period,country_code)])

    collect_poly_damages = [(line[0],line[1][0],line[1][1]) for line in collect_poly_damages]
    damaged_poly_country = power_poly.get(country_code).merge(pd.DataFrame(collect_poly_damages,columns=['return_period','index','damage']),left_index=True,right_on='index')
    damaged_poly[country_code] = damaged_poly_country
#damaged_poly

In [ ]:
# points in loop
damaged_points = {country_code:[]}
for country_code in country_codes:
    overlay_points = pd.DataFrame(overlay_hazard_assets(df_ds,power_points.get(country_code)).T,columns=['asset','hazard_point'])
    collect_point_damages = []
    for asset in tqdm(overlay_points.groupby('asset'),total=len(overlay_points.asset.unique()),desc='point damage calculation for {}'.format(country_code)):
        for return_period in return_periods:
            collect_point_damages.append([return_period,get_damage_per_asset_per_rp(asset,df_ds,power_points.get(country_code),curves,maxdam,return_period,country_code)])

    collect_point_damages = [(line[0],line[1][0],line[1][1]) for line in collect_point_damages]
    damaged_points_country = power_points.get(country_code).merge(pd.DataFrame(collect_point_damages,columns=['return_period','index','damage']),left_index=True,right_on='index')
    damaged_points_country = damaged_points_country.drop(['buffered'],axis=1)
    damaged_points[country_code] = damaged_points_country
#damaged_points

In [ ]:
# extract all data from OSM for power infrastructurepower_polyline
#reproject to epsg:3857 to have coordinate system in meters
#lines
power_lines = power_polyline(osm_path)
power_lines = power_lines.rename(columns={'power':'asset'})
power_lines['geometry'] = reproject(power_lines) 
power_lines = buffer_assets(power_lines.loc[power_lines.asset.isin(
    ['cable','minor_cable','line','minor_line'])],buffer_size=100).reset_index(drop=True)
    
#polygons
power_poly = electricity(osm_path)
power_poly['geometry'] = reproject(power_poly)
"""
#polygons
power_poly = power_polygon(osm_path)
power_poly = power_poly.rename(columns={'power':'asset'})
power_poly['geometry'] = reproject(power_poly)
#power_poly = power_poly.loc[power_poly.asset.isin(['plant','substation'])].reset_index(drop=True)

#substations polygons
### KEY:VAL IN OTHER_TAGS ###
w_list = ['substation'] # add in funtion '|'.join(w_list) if more than one key word
b_list = ['minor_distribution', 'converter', 'indoor', 'pipeline', 'gas']
power_sub = retrieve_poly_subs(osm_path,w_list,b_list)
#power_sub = retrieve_poly_subs.rename(columns={'power':'asset'})
power_sub['geometry'] = reproject(power_sub)
"""
    
#points
power_points = power_point(osm_path)
#power_points = power_points.rename(columns={'power':'asset'})
power_points['geometry'] = reproject(power_points)
power_points = buffer_assets(power_points.loc[power_points.asset.isin(
    ['power_tower','power_pole'])],buffer_size=100).reset_index(drop=True)

In [ ]:
power_lines

In [ ]:
#power_points.to_csv(os.path.join(data_path,'retrieve_data/power_points.csv'))
#power_lines.to_csv(os.path.join(data_path,'retrieve_data/power_lines.csv'))
#power_poly.to_csv(os.path.join(data_path,'retrieve_data/power_poly.csv'))

In [ ]:
# lines
overlay_lines = pd.DataFrame(overlay_hazard_assets(df_ds,power_lines).T,columns=['asset','hazard_point'])

collect_line_damages = []
for asset in tqdm(overlay_lines.groupby('asset'),total=len(overlay_lines.asset.unique()),desc='polyline damage calculation for {}'.format(country_code)):
    for return_period in return_periods:
        collect_line_damages.append([return_period,get_damage_per_asset_per_rp(asset,df_ds,power_lines,curves,maxdam,return_period,country_code)])

collect_line_damages = [(line[0],line[1][0],line[1][1]) for line in collect_line_damages]
damaged_lines = power_lines.merge(pd.DataFrame(collect_line_damages,columns=['return_period','index','damage']),left_index=True,right_on='index')
damaged_lines = damaged_lines.drop(['buffered'],axis=1) 
damaged_lines

In [ ]:
# polygons
overlay_poly = pd.DataFrame(overlay_hazard_assets(df_ds,power_poly).T,columns=['asset','hazard_point'])

collect_poly_damages = []
for asset in tqdm(overlay_poly.groupby('asset'),total=len(overlay_poly.asset.unique()),desc='polygon damage calculation for {}'.format(country_code)):
    for return_period in return_periods:
        collect_poly_damages.append([return_period,get_damage_per_asset_per_rp(asset,df_ds,power_poly,curves,maxdam,return_period,country_code)])

collect_poly_damages = [(line[0],line[1][0],line[1][1]) for line in collect_poly_damages]
damaged_poly = power_poly.merge(pd.DataFrame(collect_poly_damages,columns=['return_period','index','damage']),left_index=True,right_on='index')
damaged_poly

In [ ]:
# points
overlay_points = pd.DataFrame(overlay_hazard_assets(df_ds,power_points).T,columns=['asset','hazard_point'])

collect_point_damages = []
#return_periods = ['1_10','1_50']
for asset in tqdm(overlay_points.groupby('asset'),total=len(overlay_points.asset.unique()),desc='point damage calculation for {}'.format(country_code)):
    for return_period in return_periods:
        collect_point_damages.append([return_period,get_damage_per_asset_per_rp(asset,df_ds,power_points,curves,maxdam,return_period,country_code)])

collect_point_damages = [(line[0],line[1][0],line[1][1]) for line in collect_point_damages]
damaged_points = power_points.merge(pd.DataFrame(collect_point_damages,columns=['return_period','index','damage']),left_index=True,right_on='index')
damaged_points = damaged_points.drop(['buffered'],axis=1)

#damaged_points.to_csv(os.path.join(data_path,'retrieve_data/damaged_points.csv'))
damaged_points

In [ ]:
damaged_points
#damaged_points['LAO']

In [ ]:
# load collected power grid data
pg_data_path = os.path.join(data_path, 'pg_data')
pg_data = gpd.read_file(os.path.join(pg_data_path,"lao_line.gpkg"))
pg_data = pd.DataFrame(pg_data.copy())
print(pg_data.head())
pg_data.geometry = pygeos.from_shapely(pg_data.geometry)

pg_data['geometry'] = reproject(pg_data)
pg_data = buffer_assets(pg_data.loc[pg_data.asset.isin(['line'])],buffer_size=100).reset_index(drop=True)

In [ ]:
# lines
overlay_lines = pd.DataFrame(overlay_hazard_assets(df_ds,pg_data).T,columns=['asset','hazard_point'])
#print(overlay_lines.asset.unique())

collect_line_damages = []
for asset in tqdm(overlay_lines.groupby('asset'),total=len(overlay_lines.asset.unique()),desc='polyline damage calculation for {}'.format(country_code)):
    for return_period in return_periods:
        collect_line_damages.append([return_period,get_damage_per_asset_per_rp(asset,df_ds,pg_data,curves,maxdam,return_period,country_code)])

collect_line_damages = [(line[0],line[1][0],line[1][1]) for line in collect_line_damages]
damaged_lines = pg_data.merge(pd.DataFrame(collect_line_damages,columns=['return_period','index','damage']),left_index=True,right_on='index')
damaged_lines = damaged_lines.drop(['buffered'],axis=1)
damaged_lines